In [1]:
# Utility script

# CRLB

In [2]:
import numpy as np
from scipy.interpolate import RectBivariateSpline as rbs
from scipy.misc import central_diff_weights
# from matplotlib import pyplot as plt

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
from util_registration import similarity_transform

In [4]:
#%% 
def  eval_riVal__for_dp(gt,ci_s,F, iPar,h):
    '''
    Evaluates pixel values (by ri interpolation) for diff_im__for_crlb.
    
    Input:
        gt - geometric transformation parameters (center_x, center_y, scale, rotation),
                center_x, center_y - ci (current image) center coordinates (relative to ri)
                [rot - in degrees];
        ci_s - size of ci;
        F - ri-based interpolant;
        iPar - number of the considered gt parameter (possible values: 0,1,2,3);
        h - step for parameter grid (diff_im__for_crlb needs few values for each ci pixel for numerical defferentiation).
    
    Output:
        fVals - evaluated pixel values array [size: ci_s**2 x 5],
                number of fVals rows is equal to number ci pixels,
                number of fVals columns is equal to the number of points (5) in parameter grid.
    
    Comments:
        We consider ci to be squared.       # XXX - Attension.
    '''
    
    num_diff_grid = np.arange(-2,3)
    nP = len(num_diff_grid)
    par_grid = gt[iPar] + num_diff_grid*h
    
    nInd = ci_s**2
    fVals = np.zeros( (nInd,nP) )
    
    for ind in range(nP):
        par_ = par_grid[ind]
        gt_list = list(gt)
        gt_list[iPar] = par_
        gt_new = tuple(gt_list)
        fVals[:,ind], _, _ = similarity_transform(gt_new,ci_s, F)
    return fVals



#%% 
def  diff_im__for_crlb(ri,gt,ci_s):
    '''
    Numerically computes image derivatives at given coordinates relative to ri (using interpolation)
        [compute partial derivatives relative to "4 gt parameters"].
    
    Input:
        ri - given (reference) image;
        gt - geometric transformation parameters (center_x, center_y, scale, rotation),
                center_x, center_y - ci (current image) center coordinates (relative to ri)
                [rot - in degrees];
        ci_s - size of ci.
    
    Output:
        Px - partial derivatives (with respect to center_x) for all given coordinates
                [coordinates (ci pixels coordinates relative to ri grid) are determined by gt and ci_s];
        Py - partial derivatives (with respect to center_y) for all given coordinates;
        Ps - partial derivatives (with respect to scale) for all given coordinates;
        Pa - partial derivatives (with respect to rotation) for all given coordinates.
    
    Comments:
        We consider ri and ci be squared.       # XXX - Attension.
        Px, Py, Ps, Pa are vectors (for flattened 2D grid) of equal length.
    '''
   
    
    ri_s = np.shape(ri)[0]
    F = rbs(np.arange(ri_s), np.arange(ri_s), ri)
    
    
    h_list = [0.03, 0.03, 0.005, 0.05];     # Steps for num differentiation.
    nPar = len(h_list)
    
    nInd = ci_s**2
    P_matr = np.zeros( (nInd,nPar) )
    for iPar in range(nPar):
        h = h_list[iPar]
        fVals = eval_riVal__for_dp(gt,ci_s,F, iPar,h)
        nP = fVals.shape[1]
        
        for ind in range(nInd):
            fv = fVals[ind,:]
            P_matr[ind,iPar] = np.sum(fv * central_diff_weights(nP)) / h
            
    Px = P_matr[:,0]
    Py = P_matr[:,1]
    Ps = P_matr[:,2]
    Pa = P_matr[:,3]

    return Px, Py, Ps, Pa



#%%
def crlb__by_RI_interp(ri,gt,
                       ci_s,sigm):
    '''
    CRLB on ri evaluation
        [CRLB by Yetic, Nehorai, “Performance Bounds on Image Registration”, 2006 
         direct 2.58 implementation]
    
    Input:
        ri - reference image;
        gt - geometric transformation parameters (center_x, center_y, scale, rotation),
                center_x, center_y - ci center coordinates (relative to ri);
        ci_s - size of ci (current image);
        sigm - noise standard deviation (SD).
    
    Output:
        crlb - estimates for parameters errors SD (4 elements);
        cov_feature - covarience matrix estimate without (1/sigm**2) factor
                        (for feature creation to machine learning).
    
    Comments:
        We consider ci and ri squared.       # XXX - Attension.
    '''
  
    Px, Py, Ps, Pa  = diff_im__for_crlb(ri,gt,ci_s)
    
    
    #%%
    # For simplicity
    c = 1/sigm**2
    
    #%% Fisher matrix without coef [c = 1/sigm**2].
    Jaa = np.sum(Pa * Pa)
    Jas = np.sum(Pa * Ps)
    Jax = np.sum(Pa * Px)
    Jay = np.sum(Pa * Py)
    Jss = np.sum(Ps * Ps)
    Jsx = np.sum(Ps * Px)
    Jsy = np.sum(Ps * Py)
    Jxx = np.sum(Px * Px)
    Jyy = np.sum(Py * Py)
    Jxy = np.sum(Px * Py)
    
    
    FIM = c * np.array([ [Jxx, Jxy, Jsx, Jax],
                         [Jxy, Jyy, Jsy, Jay],
                         [Jsx, Jsy, Jss, Jas],
                         [Jax, Jay, Jas, Jaa] ]
                       )
    
    #%%
    try:
        covM = np.linalg.inv(FIM)
    except np.linalg.LinAlgError:
        print()
        print('------ Maybe singular matrix -------')
        print()
        covM = np.nan * np.ones( (4,4) )

    
    #%%
    # For feature creation to machine learning.
    cov_feature = c * covM
    
    crlb = np.sqrt( np.diag(covM) )
    return crlb, cov_feature